<a href="https://colab.research.google.com/github/Pistolll/praktika4_TOVII/blob/main/2_Word2vec_emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование векторных представлений эмотиконов

Загрузите корпус текстов, содержащих предложения как со словами, так и эмотиконами. Выведите несколько предложений на экран, убедитесь, что эмотиконы отображаются корректно. Кодировка текста - utf-8.

Разделите предложения по словам, считая эмотикон отдельным словом (нужно подобрать правильный символ-разделитель). В корпусе ровно 1000000 строк.

In [1]:
import os
from time import time
import gdown
URL='https://drive.google.com/u/1/uc?id=1TPaQTYDpf7p2EklUkLdjK9T5ATztj7oF&export=download'
gdown.download(url=URL)
!unzip corpus.zip

Downloading...
From: https://drive.google.com/u/1/uc?id=1TPaQTYDpf7p2EklUkLdjK9T5ATztj7oF&export=download
To: /content/corpus.zip
100%|██████████| 22.0M/22.0M [00:00<00:00, 130MB/s] 


Archive:  corpus.zip
replace corpus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: corpus.txt              


In [2]:
# читаем файл и разбиваем на слова по пробелу.
with open("./corpus.txt" , 'r', encoding='utf-8') as f:
            lines = f.readlines()
            train_data = [line.strip().split() for line in lines]

In [3]:
print(*train_data[:10],sep='\n') # несколько предложений

['they', 'know', 'what', 'i', 'love', '😊']
['lemme', 'get', 'to', 'school', '🙄']
['watching', 'the', 'ellen', 'degeneres', 'show', '😃']
['i', 'spent', 'twenty', 'minutes', 'herding', 'turkeys', 'by', 'myself', 'so', "that's", 'how', 'my', 'days', 'going', '🦃']
['hello', 'seattle', 'cityscape', '🏗']
['was', 'that', 'a', 'sheep', 'brain', 'in', 'your', 'snap', '👀']
['just', 'waiting', 'for', 'my', 'beard', 'to', 'connect', 'so', 'i', 'can', 'go', 'to', 'my', 'third', 'form', '🙄']
['aye', 'i', 'tried', 'my', 'best', 'to', 'get', 'him', 'out', 'loser', 'dan', '😝']
['day', 'tracking', 'our', 'next', 'ep', 'down', '👏']
['hey', "it's", 'my', 'birthday', "i'm", 'could', 'you', 'say', 'happy', 'birthday', 'flo', '🎉', 'x8']


In [4]:
len(train_data) # число слов

1000000

Эмотиконы представлены в виде символов UNICODE с определенными кодами. Функция ниже позволяет проверить, является ли символ или слово эмотиконом. Будем использовать ее.

In [5]:
# Проверка, является ли слово эмотиконом
import re
emoji_re = re.compile(
    "["
    u"\U00002712\U00002714\U00002716\U0000271d\U00002721\U00002728\U00002733\U00002734\U00002744\U00002747\U0000274c\U0000274e\U00002753-\U00002755\U00002757\U00002763\U00002764\U00002795-\U00002797\U000027a1\U000027b0\U000027bf\U00002934\U00002935\U00002b05-\U00002b07\U00002b1b\U00002b1c\U00002b50\U00002b55\U00003030\U0000303d\U0001f004\U0001f0cf\U0001f170\U0001f171\U0001f17e\U0001f17f\U0001f18e\U0001f191-\U0001f19a\U0001f201\U0001f202\U0001f21a\U0001f22f\U0001f232-\U0001f23a\U0001f250\U0001f251\U0001f300-\U0001f321\U0001f324-\U0001f393\U0001f396\U0001f397\U0001f399-\U0001f39b\U0001f39e-\U0001f3f0\U0001f3f3-\U0001f3f5\U0001f3f7-\U0001f4fd\U0001f4ff-\U0001f53d\U0001f549-\U0001f54e\U0001f550-\U0001f567\U0001f56f\U0001f570\U0001f573-\U0001f579\U0001f587\U0001f58a-\U0001f58d\U0001f590\U0001f595\U0001f596\U0001f5a5\U0001f5a8\U0001f5b1\U0001f5b2\U0001f5bc\U0001f5c2-\U0001f5c4\U0001f5d1-\U0001f5d3\U0001f5dc-\U0001f5de\U0001f5e1\U0001f5e3\U0001f5ef\U0001f5f3\U0001f5fa-\U0001f64f\U0001f680-\U0001f6c5\U0001f6cb-\U0001f6d0\U0001f6e0-\U0001f6e5\U0001f6e9\U0001f6eb\U0001f6ec\U0001f6f0\U0001f6f3\U0001f910-\U0001f918\U0001f980-\U0001f984\U0001f9c0\U00003297\U00003299\U000000a9\U000000ae\U0000203c\U00002049\U00002122\U00002139\U00002194-\U00002199\U000021a9\U000021aa\U0000231a\U0000231b\U00002328\U00002388\U000023cf\U000023e9-\U000023f3\U000023f8-\U000023fa\U000024c2\U000025aa\U000025ab\U000025b6\U000025c0\U000025fb-\U000025fe\U00002600-\U00002604\U0000260e\U00002611\U00002614\U00002615\U00002618\U0000261d\U00002620\U00002622\U00002623\U00002626\U0000262a\U0000262e\U0000262f\U00002638-\U0000263a\U00002648-\U00002653\U00002660\U00002663\U00002665\U00002666\U00002668\U0000267b\U0000267f\U00002692-\U00002694\U00002696\U00002697\U00002699\U0000269b\U0000269c\U000026a0\U000026a1\U000026aa\U000026ab\U000026b0\U000026b1\U000026bd\U000026be\U000026c4\U000026c5\U000026c8\U000026ce\U000026cf\U000026d1\U000026d3\U000026d4\U000026e9\U000026ea\U000026f0-\U000026f5\U000026f7-\U000026fa\U000026fd\U00002702\U00002705\U00002708-\U0000270d\U0000270f]|[#]\U000020e3|[*]\U000020e3|[0]\U000020e3|[1]\U000020e3|[2]\U000020e3|[3]\U000020e3|[4]\U000020e3|[5]\U000020e3|[6]\U000020e3|[7]\U000020e3|[8]\U000020e3|[9]\U000020e3|\U0001f1e6[\U0001f1e8-\U0001f1ec\U0001f1ee\U0001f1f1\U0001f1f2\U0001f1f4\U0001f1f6-\U0001f1fa\U0001f1fc\U0001f1fd\U0001f1ff]|\U0001f1e7[\U0001f1e6\U0001f1e7\U0001f1e9-\U0001f1ef\U0001f1f1-\U0001f1f4\U0001f1f6-\U0001f1f9\U0001f1fb\U0001f1fc\U0001f1fe\U0001f1ff]|\U0001f1e8[\U0001f1e6\U0001f1e8\U0001f1e9\U0001f1eb-\U0001f1ee\U0001f1f0-\U0001f1f5\U0001f1f7\U0001f1fa-\U0001f1ff]|\U0001f1e9[\U0001f1ea\U0001f1ec\U0001f1ef\U0001f1f0\U0001f1f2\U0001f1f4\U0001f1ff]|\U0001f1ea[\U0001f1e6\U0001f1e8\U0001f1ea\U0001f1ec\U0001f1ed\U0001f1f7-\U0001f1fa]|\U0001f1eb[\U0001f1ee-\U0001f1f0\U0001f1f2\U0001f1f4\U0001f1f7]|\U0001f1ec[\U0001f1e6\U0001f1e7\U0001f1e9-\U0001f1ee\U0001f1f1-\U0001f1f3\U0001f1f5-\U0001f1fa\U0001f1fc\U0001f1fe]|\U0001f1ed[\U0001f1f0\U0001f1f2\U0001f1f3\U0001f1f7\U0001f1f9\U0001f1fa]|\U0001f1ee[\U0001f1e8-\U0001f1ea\U0001f1f1-\U0001f1f4\U0001f1f6-\U0001f1f9]|\U0001f1ef[\U0001f1ea\U0001f1f2\U0001f1f4\U0001f1f5]|\U0001f1f0[\U0001f1ea\U0001f1ec-\U0001f1ee\U0001f1f2\U0001f1f3\U0001f1f5\U0001f1f7\U0001f1fc\U0001f1fe\U0001f1ff]|\U0001f1f1[\U0001f1e6-\U0001f1e8\U0001f1ee\U0001f1f0\U0001f1f7-\U0001f1fb\U0001f1fe]|\U0001f1f2[\U0001f1e6\U0001f1e8-\U0001f1ed\U0001f1f0-\U0001f1ff]|\U0001f1f3[\U0001f1e6\U0001f1e8\U0001f1ea-\U0001f1ec\U0001f1ee\U0001f1f1\U0001f1f4\U0001f1f5\U0001f1f7\U0001f1fa\U0001f1ff]|\U0001f1f4\U0001f1f2|\U0001f1f5[\U0001f1e6\U0001f1ea-\U0001f1ed\U0001f1f0-\U0001f1f3\U0001f1f7-\U0001f1f9\U0001f1fc\U0001f1fe]|\U0001f1f6\U0001f1e6|\U0001f1f7[\U0001f1ea\U0001f1f4\U0001f1f8\U0001f1fa\U0001f1fc]|\U0001f1f8[\U0001f1e6-\U0001f1ea\U0001f1ec-\U0001f1f4\U0001f1f7-\U0001f1f9\U0001f1fb\U0001f1fd-\U0001f1ff]|\U0001f1f9[\U0001f1e6\U0001f1e8\U0001f1e9\U0001f1eb-\U0001f1ed\U0001f1ef-\U0001f1f4\U0001f1f7\U0001f1f9\U0001f1fb\U0001f1fc\U0001f1ff]|\U0001f1fa[\U0001f1e6\U0001f1ec\U0001f1f2\U0001f1f8\U0001f1fe\U0001f1ff]|\U0001f1fb[\U0001f1e6\U0001f1e8\U0001f1ea\U0001f1ec\U0001f1ee\U0001f1f3\U0001f1fa]|\U0001f1fc[\U0001f1eb\U0001f1f8]|\U0001f1fd\U0001f1f0|\U0001f1fe[\U0001f1ea\U0001f1f9]|\U0001f1ff[\U0001f1e6\U0001f1f2\U0001f1fc"
    "]+",
    flags=re.UNICODE,
)
def is_emoji(word):
    if emoji_re.match(word[0]):
        return True
    else:
        return False

# Создание и обучение модели векторных представлений

Средствами библиотеки `gensim` создайте и обучите Word2Vec модель на этом наборе слов и словарь всех "слов". Размер векторных представлений - 300, прочие аргументы выбирайте самостоятельно, с учетом последующих требований. Ограничьте время обучения, чтобы оно не превосходило 5 минут. Время обучения выведите на экран.  

In [6]:
# обучение модели
!pip install numpy==1.26.4 --force-reinstall

from gensim.models import Word2Vec
model = Word2Vec(sentences=None, vector_size=300, workers=4, min_count=10, negative=5, window=5) # создаем модель
t = time()
model.build_vocab(train_data, progress_per=10000) # словарь
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
Time to build vocab: 0.06 mins


In [7]:
model.corpus_count # число слов в корпусе

1000000

In [8]:
# обучаем модель ~ 10 минут
t = time()
model.train(train_data, total_examples=model.corpus_count, epochs=20, report_delay=1) # обучаем ~ 10 мин
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 8.86 mins


In [9]:
# сохраним
model.save("word2vec5.bin")
model.wv.save_word2vec_format('word2vec5.txt')

In [10]:
# интерфейс векторных представлений
wv=model.wv

Отберите из словаря только эмотиконы. Они являются символами Unicode. Создайте и выведите на экран таблицу pandas со со столбцами, содержащими эмотиконы и их коды. Код для символа можно получить командой `ord()`. Какой размер таблицы?

In [11]:
emo_unic_symb=[x for x in wv.index_to_key if is_emoji(x)]
emo_unic_code=[ord(x) for x in emo_unic_symb]
import pandas as pd
emo_table=pd.DataFrame(list(zip(emo_unic_symb, emo_unic_code)),columns=['Emoticon','Code'])
print('Размер таблицы ',emo_table.shape)
emo_table.head()

Размер таблицы  (911, 2)


,Emoticon,Code
0,😂,128514
1,😭,128557
2,😍,128525
3,😊,128522
4,🙄,128580


Проверьте, что в словаре помимо эмотиконов есть и обычные слова, выведите на экран 10 первых токенов из словаря.

In [12]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/23069 is i
word #1/23069 is the
word #2/23069 is to
word #3/23069 is you
word #4/23069 is a
word #5/23069 is 😂
word #6/23069 is my
word #7/23069 is and
word #8/23069 is me
word #9/23069 is for


# Эксперименты

## 1. Похожесть эмотиконов
Посчитайте и выведите на экран 10 наиболее похожих токенов на эмотикон 😊. Сколько среди них слов, сколько эмотиконов?

*Чек-поинт 1: Модель должна быть обучена так, чтобы среди похожих токенов эмотиконов было не менее 5.*

In [13]:
def count_emoji(sentence):
  e_count=len([x[0] for x in sentence if is_emoji(x[0])])
  return e_count, len(sentence)-e_count

In [14]:
query_emo=['😊'] #  🐱
#query_emo=['👀'] #  👀😭
#query_emo=['😭']
emo_similar=wv.most_similar(positive=query_emo,topn=10)
emo_count=count_emoji(emo_similar)
print('Emoticons \ words',emo_count)
print(*emo_similar,sep='\n')

Emoticons \ words (9, 1)
('😉', 0.5202940702438354)
('😀', 0.5094165802001953)
('🙂', 0.4810003936290741)
('😃', 0.4675767719745636)
('😄', 0.46650177240371704)
('😌', 0.4471561014652252)
('😁', 0.44031384587287903)
('😇', 0.4366243779659271)
('👍', 0.3766186833381653)
('and', 0.372602254152298)


## 2. Похожесть слов на эмотиконы
Посчитайте и выведите на экран 10 наиболее похожих токенов на слово `kitty`. Сколько среди них слов, сколько эмотиконов?

*Чек-поинт 2: Модель должна быть обучена так, чтобы среди похожих токенов был хотя бы 1 эмотикон.*

In [15]:
query_word=['kitty']
emo_similar=wv.most_similar(positive=query_word,topn=10)
emo_count=count_emoji(emo_similar)
print('Emoticons \ words',emo_count)
print(*emo_similar,sep='\n')


Emoticons \ words (2, 8)
('kitties', 0.545719563961029)
('kitten', 0.49815553426742554)
('cat', 0.4872535169124603)
('meow', 0.47681599855422974)
('pupper', 0.43729203939437866)
('🐈', 0.43161293864250183)
('pup', 0.4227438271045685)
('hamster', 0.4058762192726135)
('🐱', 0.4034464657306671)
('scaredy', 0.3950367867946625)


Среди похожих токенов из п.2 есть хотя бы 1 эмотикон. Найдите его, для него выведите на экран 5 самых близких токенов. Какой токен оказался самым близким к этому эмотикону, это слово или эмотикон?

In [16]:
emo_first=[x[0] for x in emo_similar if is_emoji(x[0])][0]
print(emo_first)
emo_similar_first=wv.most_similar(positive=emo_first,topn=5)
emo_count_first=count_emoji(emo_similar_first)
print('Emoticons \ words',emo_count_first)
print(*emo_similar_first,sep='\n')
print('Nearest  is a "', emo_similar_first[0][0],'" emoticon' if is_emoji(emo_similar_first[0][0]) else '" word')

🐈
Emoticons \ words (3, 2)
('🐩', 0.460335373878479)
('🐕', 0.4563700258731842)
('🐱', 0.45205044746398926)
('kitties', 0.4500630497932434)
('cat', 0.44573646783828735)
Nearest  is a " 🐩 " emoticon


## 3. Отношения эмотиконов и слов

### А)
Найдите ближайшее **слово** Х подходящее под уравнение:

🐱 - 🐶 = cat - X

и не совпадающее ни с одним из исходных токенов, входящих в уравнение.

Выведите на экран это слово и уровень похожести.

In [17]:
tokenA='🐱'
tokenB='🐶'
tokenC='cat'

out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
#candidate=[x for x in wv.similar_by_vector(out,topn=50)]
res=[x for x in candidate if not is_emoji(x[0])][0]
print(res)


('puppy', 0.470476359128952)


### Б)
 Найдите ближайший **эмотикон** Х подходящий под уравнение:

kitty - puppy = 🐱 - X  

и не совпадающий ни с одним из исходных токенов, входящих в уравнение.

Выведите на экран этот эмотикон и уровень похожести.

In [18]:
tokenA='kitty'
tokenB='puppy'
tokenC='🐱'

out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
res=[x for x in candidate if is_emoji(x[0])][0]
print(res)



('🐶', 0.32287806272506714)


In [19]:
print(emo_unic_symb)

['😂', '😭', '😍', '😊', '🙄', '😩', '👀', '💀', '😘', '💕', '👍', '🎶', '❤', '👌', '💯', '🙃', '🎉', '🔥', '😒', '👏', '👉', '😉', '😅', '💖', '😎', '😁', '😔', '💋', '😌', '🎃', '💙', '😢', '📷', '😏', '👻', '🎧', '💜', '😴', '😳', '🙏', '🌚', '😑', '😕', '💗', '😜', '👑', '💞', '🙂', '🙌', '😋', '😐', '😈', '😞', '✨', '😪', '🙈', '👇', '💔', '😄', '🌸', '🌹', '😛', '😀', '😬', '💓', '👅', '💛', '🏽', '👊', '😇', '💘', '😫', '🏻', '😱', '😆', '🏼', '🏾', '🌟', '🎈', '💪', '😤', '😡', '🎥', '🎂', '🏆', '😝', '😃', '💚', '🎤', '🍻', '💥', '👈', '🎵', '🌞', '✔', '😷', '😻', '💃', '🍃', '🐶', '🏈', '👋', '🤔', '💰', '🏀', '🔴', '😣', '🐐', '💦', '🎄', '🍾', '😓', '🌎', '😥', '🙊', '📲', '🌊', '😚', '‼', '🌻', '☺', '💫', '🌙', '🍷', '👯', '💎', '🍀', '🔘', '🍕', '😹', '🚨', '🍂', '😶', '➡', '🙁', '🎁', '👄', '😖', '💩', '✌', '🌺', '💍', '🗣', '⚠', '👎', '🐱', '🍁', '👽', '🎯', '💟', '😰', '✅', '🎀', '🎬', '🍒', '🍑', '💤', '📢', '🐾', '💸', '💝', '👂', '🌈', '🎊', '📹', '🍭', '🍸', '🍎', '🐍', '🤘', '▶', '🌴', '📍', '🎼', '🌷', '🎆', '🍹', '👫', '🌠', '😙', '😠', '🏃', '🐻', '❌', '🏡', '🤗', '🍴', '🚮', '❗', '🍦', '🍓', '😟', '🍆', '💅', '🎅', '🍰', '👐', '💨', '😮', '📸',

### В)
 Найдите ближайший **эмотикон** Х подходящий под уравнение:

 man - 😊 = cat - X  

и не совпадающий ни с одним из исходных токенов, входящих в уравнение.

Выведите на экран этот эмотикон и уровень похожести.

In [20]:
tokenA='man'
tokenB='😊' #
#tokenB='😭'
tokenC='cat'
#tokenC='dog'

out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
res=[x for x in candidate if is_emoji(x[0])][0:2]
print(res)


[('😸', 0.3597041666507721), ('😃', 0.3290247321128845)]


Вариант Г

coffee - ☕ = tired - X

In [21]:
tokenA = 'coffee'
tokenB = '☕'
tokenC = 'tired'

out = wv[tokenC] - wv[tokenA] + wv[tokenB]
candidate = [x for x in wv.similar_by_vector(out, topn=50) if x[0] not in [tokenA, tokenB, tokenC]]
res = [x for x in candidate if is_emoji(x[0])][:1]

print(res)


[('⚗', 0.35499680042266846)]


happy - 😀 = cold - X


In [22]:
tokenA = 'happy'
tokenB = '😀'
tokenC = 'cold'

out = wv[tokenC] - wv[tokenA] + wv[tokenB]
candidate = [x for x in wv.similar_by_vector(out, topn=50) if x[0] not in [tokenA, tokenB, tokenC]]
res = [x for x in candidate if is_emoji(x[0])][:1]
print(res)


[('😣', 0.29037338495254517)]


In [ ]:
fire - 🔥 = ice - X


In [23]:
tokenA = 'fire'
tokenB = '🔥'
tokenC = 'ice'

out = wv[tokenC] - wv[tokenA] + wv[tokenB]
candidate = [x for x in wv.similar_by_vector(out, topn=50) if x[0] not in [tokenA, tokenB, tokenC]]
res = [x for x in candidate if is_emoji(x[0])][:1]
print(res)


[('❄', 0.2195388674736023)]
